# 3 Predictive Analytics: 
Future demand is a key factor that will steer operational decision making of a shared rental network. As a data scientist it is your responsibility to facilitate this type of decision support. For the purpose of this assignment we will be interested in forecasting total system-level demand in the next hour. To do so, develop a prediction model that predicts bike rental demand as a function of suitable features available in or derived from the datasets (incl. the weather data).

1. Feature Engineering: Develop a rich set of features that you expect to be correlated with your target. In this process you can draw on your domain knowledge and/or conduct additional research around the topic of demand prediction in vehicle rental networks. Justify your selection of features.

2. Model Building: Select three regression algorithms that are suitable for the prediction task at hand. Explain and justify why you selected the three algorithms and describe their respective advantages and drawbacks.

3. Model Evaluation: How well do the models perform? Evaluate and benchmark your models’ performance using suitable evaluation metrics. Which model would you select for deployment?

4. Outlook: How could the selected model be improved further? Explain some of the improvement levers that you might focus on in a follow-up project.

Notes and tipps

- Make generous use of visualization techniques to clearly illustrate your findings and present them in an appealing fashion.
- Evaluate your methodology and clearly state why you have opted for a specific approach in your analysis.
- Relate your findings to the real world and interpret them for non-technical audiences (e.g. What do the coefficients in your regression model mean?, What does the achieved error mean for your model?, etc.)
- Make sure to clearly state the implications (i.e. the ”so what?”) of your findings for managers/decision makers.

## 3.1 Import

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

Import of data

In [3]:
df_trips = pd.read_csv('boston_2017_cleaned_trips.csv')
df_weather = pd.read_csv('weather_2017_cleaned.csv')

## 3.2 Feature Engineering

In advance of building the models suitable variables are enginered that help the prediction. 

In [4]:
df_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310760 entries, 0 to 1310759
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   start_time          1310760 non-null  object 
 1   end_time            1310760 non-null  object 
 2   start_station_id    1310760 non-null  int64  
 3   end_station_id      1310760 non-null  int64  
 4   start_station_name  1310760 non-null  object 
 5   end_station_name    1310760 non-null  object 
 6   bike_id             1310760 non-null  int64  
 7   user_type           1310760 non-null  object 
 8   start_hour          1310760 non-null  int64  
 9   weekday             1310760 non-null  int64  
 10  duration            1310760 non-null  float64
 11  month               1310760 non-null  int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 120.0+ MB


Our pre-processed data already contains various information about the temporal patterns e.g. the start time furthermore the hour, weekday and month of the trips.

In [5]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   max_temp   8760 non-null   float64
 1   min_temp   8760 non-null   float64
 2   precip     8760 non-null   float64
 3   date_time  8760 non-null   object 
dtypes: float64(3), object(1)
memory usage: 273.9+ KB
